### Raw data processing

In [270]:
import csv
import pandas as pd
from IPython.display import display

def read_transactions():
    with open('transactions.csv', mode='r', newline='') as file:
        # Create a CSV reader object
        csv_reader = csv.reader(file)

        # Skip the header row
        next(csv_reader)
        
        # Create a list to store the transactions
        transactions = []

        # Iterate over each row in the CSV file
        for row in csv_reader:
            # Clean all rows
            row = [cell.strip() for cell in row]

            # Create a transaction
            transaction = {}
            transaction["date"] = row[0]
            transaction["type"] = row[1].lower()
            transaction["product_type"] = row[2]
            transaction["product_name"] = row[3]
            transaction["product_id"] = row[4].upper()
            transaction["units"] = int(row[5]) if row[5] else None
            transaction["total"] = abs(float(row[6])) if row[6] else None
            
            # Append the transction to the list
            transactions.append(transaction)

    transactions = process_transactions(transactions)

    return transactions

def process_transactions(transactions):
    # Create a dictionary to store the total sales for each transaction type
    processed_transactions = {}

    # Iterate over each transaction in the list
    for transaction in transactions:
        # copy all data from transaction to new object
        pt = transaction.copy()

        # Do manual manipulation for specific transaction types
        if transaction["type"] == "buy" or transaction["type"] == "sell":
            pt["price_per_unit"] = transaction["total"] / transaction["units"]
        else:
            pt = transaction

        # Check if the transaction type is already in the dictionary
        if transaction["type"] in processed_transactions:
            # Add the processed transaction to the list
            processed_transactions[pt["type"]].append(pt)
        else:
            # Create a new key-value pair
            processed_transactions[pt["type"]] = [pt]

    return processed_transactions

# Read transactions
transactions = read_transactions()

In [271]:
realtime_stock_prices = {
    "VDHG": 65.63,
    "VGS": 127.30,
    "VAS": 100.06
}

total_years = 4

summary = {}
# buys and sells
summary["buy_total"] = 0
summary["sell_total"] = 0
# deposits and withdrawals
summary["deposit_total"] = 0
summary["withdrawal_total"] = 0
# distributions
summary["distribution_total"] = 0
# stocks
summary["stock_total"] = 0
summary["stock_real_total"] = 0
summary["stocks"] = {}
# interest
summary["interest_total"] = 0
# fees
summary["fees_total"] = 0

# process transactions to create summaries
for key, value in transactions.items():
    for transaction in value:
        transaction_type = transaction["type"]
        transaction_total = transaction["total"]
        transaction_units = transaction["units"]
        product_id = transaction["product_id"]
        product_name = transaction["product_name"]

        # handle summary totals
        if transaction_type == "buy":
            summary["buy_total"] += transaction_total
        elif transaction_type == "sell":
            summary["sell_total"] += transaction_total
        elif transaction_type == "deposit":
            summary["deposit_total"] += transaction_total
        elif transaction_type == "withdrawal":
            summary["withdrawal_total"] += transaction_total
        elif transaction_type == "distribution":
            summary["distribution_total"] += transaction_total
        elif transaction_type == "interest":
            summary["interest_total"] += transaction_total
        elif transaction_type == "fees":
            summary["fees_total"] += transaction_total

        # handle stocks
        if transaction_type == "buy" or transaction_type == "sell":
            if product_id in summary["stocks"]:
                if transaction_type == "buy":
                    summary["stocks"][product_id]["units"] += transaction_units
                    summary["stocks"][product_id]["total"] += transaction_total
                elif transaction_type == "sell":
                    summary["stocks"][product_id]["units"] -= transaction_units
                    summary["stocks"][product_id]["total"] -= transaction_total
            else:
                summary["stocks"][product_id] = {}
                summary["stocks"][product_id]["product_name"] = product_name
                if transaction_type == "buy":
                    summary["stocks"][product_id]["units"] = transaction_units
                    summary["stocks"][product_id]["total"] = transaction_total
                elif transaction_type == "sell":
                    summary["stocks"][product_id]["units"] = -transaction_units
                    summary["stocks"][product_id]["total"] = -transaction_total

# process stocks
for key, value in summary["stocks"].items():
    summary["stock_total"] += value["total"]
    summary["stock_real_total"] += realtime_stock_prices[key] * value["units"]
    # real total value of stock
    value["total_real"] = realtime_stock_prices[key] * value["units"]
    value["total_diff"] = value["total_real"] - value["total"]
    # price per unit
    value["ppu"] = value["total"] / value["units"]
    value["ppu_real"] = realtime_stock_prices[key]
    value["ppu_diff"] = value["ppu_real"] - value["ppu"]

In [272]:
print("--- Money in and out ---")
print(f"In: ${summary["deposit_total"]:.2f}")
print(f"Out: ${summary["withdrawal_total"]:.2f}")
total_account_invested = summary["deposit_total"] - summary["withdrawal_total"]
print(f">>> Total balance: ${total_account_invested:.2f}")

print("\n--- Accumulated payouts, interest and fees ---")
print(f"Dividend payouts: ${summary['distribution_total']:.2f}")
print(f"Interest payments: ${summary['interest_total']:.2f}")
print(f"Fees: ${summary['fees_total']:.2f}")
total_accumulated_gains = summary["distribution_total"] + summary["interest_total"] - summary["fees_total"]
print(f">>> Total accumulated gains: ${total_accumulated_gains:.2f}")

total_bottom_line = total_account_invested + total_accumulated_gains
print(f"\n>>> Total bottom line: ${total_bottom_line:.2f}")

print("\n--- Stocks ---")
print(f"Invested: ${summary['stock_total']:.2f}")
print(f"Real value: ${summary['stock_real_total']:.2f}")
total_stock_gains = summary["stock_real_total"] - summary["stock_total"]
print(f">>> Total stock gains: ${total_stock_gains:.2f}")

print("\n--- Portfolio ---")
total_portfolio_value = total_bottom_line + total_stock_gains
total_available_cash = total_portfolio_value - summary["stock_real_total"]
print(f"Available cash: ${total_available_cash:.2f}")
print(f"Stock value: ${summary['stock_real_total']:.2f}")
print(f">>> Total portfolio value: ${total_portfolio_value:.2f}")

total_gains = total_portfolio_value - total_account_invested
percentage_increase = (total_gains / total_account_invested) * 100
print(f"\n>>> Total gains: ${total_gains:.2f} ({percentage_increase:.2f}%)")
print(f">>> Total gains (over {total_years} years): ${total_gains / total_years:.2f} ({percentage_increase / total_years:.2f}%)")

--- Money in and out ---
In: $49211.00
Out: $1079.70
>>> Total balance: $48131.30

--- Accumulated payouts, interest and fees ---
Dividend payouts: $2244.53
Interest payments: $2.60
Fees: $72.00
>>> Total accumulated gains: $2175.13

>>> Total bottom line: $50306.43

--- Stocks ---
Invested: $50256.48
Real value: $55109.28
>>> Total stock gains: $4852.80

--- Portfolio ---
Available cash: $49.95
Stock value: $55109.28
>>> Total portfolio value: $55159.23

>>> Total gains: $7027.93 (14.60%)
>>> Total gains (over 4 years): $1756.98 (3.65%)
